In [154]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression

In [161]:
float_type = 'float64'
feature = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Master', 'Aristocratic']

def df_feature(df):
    df['Sex'] = (df['Sex'] == 'male')
    title_feature(df)
    df[feature] = df[feature].astype(float_type)
    fillna(df)

def fillna(df):
    df.Age = df.Age.fillna(round(df.Age.mean()))
    fare_groupby = df[df.Fare > 0.0].groupby('Pclass')['Fare'].mean()
    cond = (df.Fare == 0.0) | (df.Fare.isna())
    df.loc[cond, 'Fare'] = fare_groupby[df.loc[cond, 'Pclass']].values
    df.loc[df.Embarked.isnull(), 'Embarked'] = 'S'

def title_feature(df):
    df['Title'] = df.apply(lambda row: re.search('\w[\s\w]+', re.search(',\s+[\s\w]+\.', row['Name']).group(0)).group(0), axis=1).astype(str)
    df.loc[df.Title == 'Mlle', 'Title'] = 'Miss'
    df.loc[df.Title == 'Monsieur', 'Title'] = 'Mr'
    df.loc[(df.Title == 'Ms') | (df.Title == 'Mme'), 'Title'] = 'Mrs'
    df.loc[~df.Title.isin(['Miss', 'Mr', 'Mrs', 'Master']), 'Title'] = 'Aristocratic'
    df['Aristocratic'] = (df.Title == 'Aristocratic').astype(float_type)
    df['Master'] = (df.Title == 'Master').astype(float_type)

In [159]:
def submission(train_path, test_path):
    df_train = pd.read_csv(train_path, index_col='PassengerId')
    df_test = pd.read_csv(test_path, index_col='PassengerId')
    df = df_train.append(df_test, verify_integrity=True, sort=False)
    df_feature(df)
    df_train = df.loc[df_train.index]
    df_test = df.loc[df_test.index]
    df_test.drop(columns='Survived', inplace=True)
    model = LogisticRegression(max_iter=256)
    model.fit(df_train[feature], df_train.Survived)
    df_test['Survived'] = model.predict(df_test[feature])
    return df_test.Survived.astype(int)

In [160]:
submission_path = 'submission.csv'
df_submission = submission('./train.csv', './test.csv')
df_submission.to_csv(submission_path, header=True)

In [267]:
"""
#df.Sex.get_dummies()
#pd.get_dummies(df, columns)

df = pd.read_csv('./titanic/train.csv', index_col='PassengerId')
x = np.array(df.Cabin.astype('str'))
vectorize_len = np.vectorize(len)
y = vectorize_len(x)
x[(x != 'nan')]
pd.crosstab(df['Cabin'], df['nametype'])

plt.figure(figsize=(15,35))
plt.title("Dataset visualization")
plt.scatter(df.Sex, df.Age, c=df.Survived, cmap=ListedColormap(["blue", "red"]), s=20)
plt.show()
"""

'\n#df.Sex.get_dummies()\n#pd.get_dummies(df, columns)\n\ndf = pd.read_csv(\'./titanic/train.csv\', index_col=\'PassengerId\')\nx = np.array(df.Cabin.astype(\'str\'))\nvectorize_len = np.vectorize(len)\ny = vectorize_len(x)\nx[(x != \'nan\')]\npd.crosstab(df[\'Cabin\'], df[\'nametype\'])\n\nplt.figure(figsize=(15,35))\nplt.title("Dataset visualization")\nplt.scatter(df.Sex, df.Age, c=df.Survived, cmap=ListedColormap(["blue", "red"]), s=20)\nplt.show()\n'

In [266]:
"""
df = pd.read_csv('./titanic/train.csv')#, index_col='PassengerId')
#df_feature(df)
df['Surname'] = df.apply(lambda row: re.search("[\w\s']+", row['Name']).group(0), axis=1).astype(str)
count = df[['PassengerId', 'Surname', 'Survived']].groupby('Surname').count()
count[count.PassengerId != count.Survived]

test = pd.read_csv('./titanic/test.csv')#, index_col='PassengerId')
test['Surname'] = test.apply(lambda row: re.search("[\w\s']+", row['Name']).group(0), axis=1).astype(str)
set(test['Surname']) & set(df['Surname'])#.value_counts()
"""

'\ndf = pd.read_csv(\'./titanic/train.csv\')#, index_col=\'PassengerId\')\n#df_feature(df)\ndf[\'Surname\'] = df.apply(lambda row: re.search("[\\w\\s\']+", row[\'Name\']).group(0), axis=1).astype(str)\ncount = df[[\'PassengerId\', \'Surname\', \'Survived\']].groupby(\'Surname\').count()\ncount[count.PassengerId != count.Survived]\n\ntest = pd.read_csv(\'./titanic/test.csv\')#, index_col=\'PassengerId\')\ntest[\'Surname\'] = test.apply(lambda row: re.search("[\\w\\s\']+", row[\'Name\']).group(0), axis=1).astype(str)\nset(test[\'Surname\']) & set(df[\'Surname\'])#.value_counts()\n'